In [4]:
import httpx
from datetime import timedelta
from prefect import flow, task
from prefect.tasks import task_input_hash


@task(cache_key_fn=task_input_hash, cache_expiration=timedelta(hours=1),name="t任务")
def get_url(url: str, params: dict = None):
    response = httpx.get(url, params=params)
    response.raise_for_status()
    return response.json()


def get_open_issues(repo_name: str, open_issues_count: int, per_page: int = 100):
    issues = []
    pages = range(1, -(open_issues_count // -per_page) + 1)
    for page in pages:
        issues.append(
            get_url(
                f"https://api.github.com/repos/{repo_name}/issues",
                params={"page": page, "per_page": per_page, "state": "open"},
            )
        )
    return [i for p in issues for i in p]


@flow(retries=3, retry_delay_seconds=5, log_prints=True,name="f流")
def get_repo_info(repo_name: str = "PrefectHQ/prefect"):
    repo_stats = get_url(f"https://api.github.com/repos/{repo_name}")
    issues = get_open_issues(repo_name, repo_stats["open_issues_count"])
    issues_per_user = len(issues) / len(set([i["user"]["id"] for i in issues]))
    print(f"{repo_name} repository statistics 🤓:")
    print(f"Stars 🌠 : {repo_stats['stargazers_count']}")
    print(f"Forks 🍴 : {repo_stats['forks_count']}")
    print(f"Average open issues per user 💌 : {issues_per_user:.2f}")


if __name__ == "__main__":
    get_repo_info()


11:29:54.073 | INFO    | prefect.engine - Created flow run 'enormous-bonobo' for flow 'f流'

11:29:54.133 | INFO    | Flow run 'enormous-bonobo' - Created task run 't任务-0' for task 't任务'

11:29:54.135 | INFO    | Flow run 'enormous-bonobo' - Executing 't任务-0' immediately...

11:29:54.228 | INFO    | Task run 't任务-0' - Finished in state Cached(type=COMPLETED)

11:29:54.301 | INFO    | Flow run 'enormous-bonobo' - Created task run 't任务-1' for task 't任务'

...

11:29:55.017 | INFO    | Flow run 'enormous-bonobo' - PrefectHQ/prefect repository statistics 🤓:

11:29:55.018 | INFO    | Flow run 'enormous-bonobo' - Stars 🌠 : 13921

11:29:55.020 | INFO    | Flow run 'enormous-bonobo' - Forks 🍴 : 1424

11:29:55.021 | INFO    | Flow run 'enormous-bonobo' - Average open issues per user 💌 : 2.18

11:29:55.048 | INFO    | Flow run 'enormous-bonobo' - Finished in state Completed('All states completed.')